MLogit Training
===

Test notebook for using R on MSI to train the mlogit models.

In [1]:
install.packages("knitr")
install.packages("car")
# these are needed by the helper R file
install.packages("ggplot2")
install.packages("latex2exp")
install.packages("mlogit")
install.packages("stringr")
install.packages("parallel")
install.packages("tidyr")
install.packages("readr")
install.packages("scales")
install.packages("dplyr")
install.packages("gridExtra")

Warning message in install.packages("knitr"):
“installation of package ‘knitr’ had non-zero exit status”
also installing the dependency ‘dfidx’


Warning message:
“package ‘parallel’ is not available (for R version 3.6.2)”
Warning message:
“package ‘parallel’ is a base package, and should not be updated”
also installing the dependencies ‘rlang’, ‘tidyselect’, ‘vctrs’


Warning message in install.packages("tidyr"):
“installation of package ‘rlang’ had non-zero exit status”
Warning message in install.packages("tidyr"):
“installation of package ‘vctrs’ had non-zero exit status”
Warning message in install.packages("tidyr"):
“installation of package ‘tidyselect’ had non-zero exit status”
Warning message in install.packages("tidyr"):
“installation of package ‘tidyr’ had non-zero exit status”
Warning message in install.packages("readr"):
“installation of package ‘readr’ had non-zero exit status”
also installing the dependency ‘farver’


Warning message in install.packages("scales"):
“installa

In [2]:
library(knitr)
knitr::opts_chunk$set(echo=F, warning=F, cache=T, message=F, fig.width=6, fig.height=2.5)
source("helper.R")
mutate = dplyr::mutate
library(car)

Loading required package: dfidx


Attaching package: ‘dfidx’


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘scales’


The following object is masked from ‘package:readr’:

    col_factor



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode




In [3]:
citation("mlogit")


To cite package ‘mlogit’ in publications use:

  Yves Croissant (2020). mlogit: Multinomial Logit Models. R package
  version 1.1-0. https://CRAN.R-project.org/package=mlogit

A BibTeX entry for LaTeX users is

  @Manual{,
    title = {mlogit: Multinomial Logit Models},
    author = {Yves Croissant},
    year = {2020},
    note = {R package version 1.1-0},
    url = {https://CRAN.R-project.org/package=mlogit},
  }


In [5]:
fn <- '/home/lana/shared/caringbridge/data/projects/sna-social-support/author_initiations/author_initiation_choices_train_all_nofuturepatient.csv'
DM <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddi')
#%>%
#mutate(
#    is_friend_of_friend=is_friend_of_friend - is_reciprocal    #ifelse(is_friend_of_friend & -is_reciprocal, 1, 0)
#)
#%>%
#    mutate(
#        has_indeg=ifelse(indeg > 0, 1, 0),
#        has_outdeg=ifelse(outdeg > 0, 1, 0)
#      )
# this mutate can be used to construct other features, if desired

In [6]:
# is_friend_of_friend is computed as being EITHER a reciprocated tie OR a strong connection via a friend
DM$is_friend_of_friend = DM$is_friend_of_friend - DM$is_reciprocal

In [7]:
#hcf = health condition factor
DM$hcf = relevel(factor(DM$target_health_condition), ref="None")

In [8]:
nrow(DM); nrow(DM) / 25

[1] 3878525

[1] 155141

In [7]:
# load the test data
fn <- '/home/lana/shared/caringbridge/data/projects/sna-social-support/author_initiations/author_initiation_choices_test_all_nofuturepatient.csv'
DM_test <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddi')
DM_test$is_friend_of_friend = DM_test$is_friend_of_friend - DM_test$is_reciprocal
DM_test$hcf = relevel(factor(DM_test$target_health_condition), ref="None")
H_test <- DM_test %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')


Warning message:
“Setting row names on a tibble is deprecated.”


In [9]:
head(DM)

choice_id,initiator_user_id,candidate_user_id,is_target,target_outdegree,target_indegree,target_has_indegree,is_reciprocal,is_weakly_connected,is_friend_of_friend,⋯,target_health_condition,is_health_condition_shared,target_is_multisite_author,target_is_mixedsite_author,target_update_count,target_update_frequency,target_days_since_most_recent_update,target_days_since_first_update,target_site_visits,hcf
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<fct>
0,15878504,5758186,1,10,41,1,0,1,0,⋯,Cancer,1,0,0,628,10.271729,0.0000,1118.324664,118271,Cancer
0,15878504,8745849,0,0,1,1,0,1,0,⋯,Cardiovascular/Stroke,0,0,0,17,2.249373,1132.4663,226.729861,1962,Cardiovascular/Stroke
0,15878504,7054610,0,3,3,1,0,1,0,⋯,Neurological Condition,0,0,0,11,5.774769,1446.3149,57.145139,440,Neurological Condition
0,15878504,24619441,0,0,0,0,0,0,0,⋯,Other,0,0,0,5,40.986717,322.2351,3.659722,161,Other
0,15878504,12664705,0,0,0,0,0,0,0,⋯,Cancer,1,0,0,17,1.863397,504.8274,273.693750,2765,Cancer
0,15878504,3604528,0,0,0,0,0,0,0,⋯,None,0,0,0,8,7.619263,1977.1469,31.499109,631,None


In [10]:
# column names
names(DM)

[1] "choice_id"                           
 [2] "initiator_user_id"                   
 [3] "candidate_user_id"                   
 [4] "is_target"                           
 [5] "target_outdegree"                    
 [6] "target_indegree"                     
 [7] "target_has_indegree"                 
 [8] "is_reciprocal"                       
 [9] "is_weakly_connected"                 
[10] "is_friend_of_friend"                 
[11] "is_author_type_shared"               
[12] "target_author_type"                  
[13] "initiator_author_type"               
[14] "target_health_condition"             
[15] "is_health_condition_shared"          
[16] "target_is_multisite_author"          
[17] "target_is_mixedsite_author"          
[18] "target_update_count"                 
[19] "target_update_frequency"             
[20] "target_days_since_most_recent_update"
[21] "target_days_since_first_update"      
[22] "target_site_visits"                  
[23] "hcf"

In [11]:
H_train <- DM %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')


In [11]:
# DO NOT RUN THIS CELL - DEPRECATED IMPLEMENTATION

examples <- DM %>% distinct(choice_id) %>% mutate(r=runif(nrow(DM %>% distinct(choice_id)))) %>% arrange(r)


H_train <- DM %>% inner_join(head(examples, 150000), by='choice_id') %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
#     select(choice_id, alt_id, is_target, target_indegree, target_has_indegree) %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')

H_valid <- DM %>% inner_join(tail(examples, 5141), by='choice_id') %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
#     select(choice_id, alt_id, y, outdeg:has_outdeg) %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')


Warning message:
“Setting row names on a tibble is deprecated.”
Warning message:
“Setting row names on a tibble is deprecated.”


In [12]:
head(H_train)

# A tibble: 3,878,525 x 25
   choice_id initiator_user_… candidate_user_… is_target target_outdegree
 *     <int>            <int>            <int> <lgl>                <int>
 1         0         15878504          5758186 TRUE                    10
 2         0         15878504          8745849 FALSE                    0
 3         0         15878504          7054610 FALSE                    3
 4         0         15878504         24619441 FALSE                    0
 5         0         15878504         12664705 FALSE                    0
 6         0         15878504          3604528 FALSE                    0
 7         0         15878504         10219933 FALSE                    1
 8         0         15878504          2013798 FALSE                    0
 9         0         15878504          3625288 FALSE                    1
10         0         15878504          2890036 FALSE                    0
# … with 3,878,515 more rows, and 21 more variables: target_indegree <int>,
#   targe

In [13]:
nums <- unlist(lapply(H_train, is.numeric))  
#cor(H_train[ , nums])
#unlist(lapply(nums, "_id"))
#select(H_train,contains("_id"))
cor(dplyr::select_if(H_train, is.numeric) %>% select(-contains("_id")))

ERROR: Error in cor(dplyr::select_if(H_train, is.numeric) %>% select(-contains("_id"))): 'x' must be numeric


In [ ]:
start_time <- proc.time()
fs <- list(
#  mlogit(is_target ~ censored_log(target_outdegree, min=1) + censored_log(target_indegree, min=1) + is_reciprocal | 0, H_train),
  mlogit(is_target ~ factor(target_author_type) 
         + factor(target_author_type) : is_author_type_shared 
         + is_author_type_shared | 0, H_train),
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
#         + target_site_visits
         | 0, H_train)
)
total_time <- proc.time() - start_time
total_time

In [14]:
stargazer::stargazer(
  fs,
  dep.var.caption = "",
  #covariate.labels = c("log outdegree", "Has outegree", "log indegree", "Has indegree",
   #                    "is shared condition",
    #                   "is patient authored", "is mixed authored", "is shared authored",
     #                  "visit count"),
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                               is_target          
                                                           (1)            (2)     
----------------------------------------------------------------------------------
censored_log(target_outdegree, min = 1)                                -0.191***  
                                                                        (0.005)   
                                                                                  
target_has_indegree                                                    0.756***   
                                                                        (0.017)   
                                                                                  
censored_log(target_indegree, min = 1)                                 0.649***   
                                                                        (0.005)   
                                                                                  
is_

In [15]:
print(paste("Train accuracy  &  ",
    lapply(fs, function(x) acc(x, H_train)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))
print(paste("Test accuracy   &  ",
    lapply(fs, function(x) acc(x, H_test)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))


[1] "Train accuracy  &   0.0462 & 0.7398  \\"
[1] "Test accuracy   &   0.0452 & 0.7727  \\"


In [16]:
# % increase in the odds of being the target author
# "The exponential beta coefficient represents the change in the odds
# of the dependent variable being in a particular category vis-a-vis the reference category,
# associated with a one unit change of the corresponding independent variable."
(exp(fs[[2]]$coefficients) - 1) * 100

censored_log(target_outdegree, min = 1)                     target_has_indegree 
                          -1.737017e+01                            1.129328e+02 
 censored_log(target_indegree, min = 1)                           is_reciprocal 
                           9.134548e+01                            4.928266e+10 
                    is_weakly_connected                     is_friend_of_friend 
                           4.853435e+02                            1.840123e+04 
        factor(target_author_type)mixed             factor(target_author_type)p 
                           2.043675e+00                           -2.152594e+01 
                  is_author_type_shared                 is_heh_condition_shared 
                           3.485179e+01                            2.370014e+01 
             target_is_multisite_author              target_is_mixedsite_author 
                           3.701897e+01                            6.065998e+01 
                    target_update_count                 target_update_frequency 
                          -3.418244e-02                            7.114286e-01 
   target_days_since_most_recent_update          target_days_since_first_update 
                          -1.127157e+00                           -1.178046e-01 
attr(,"names.sup.coef")
character(0)
attr(,"fixed")
censored_log(target_outdegree, min = 1)                     target_has_indegree 
                                  FALSE                                   FALSE 
 censored_log(target_indegree, min = 1)                           is_reciprocal 
                                  FALSE                                   FALSE 
                    is_weakly_connected                     is_friend_of_friend 
                                  FALSE                                   FALSE 
        factor(target_author_type)mixed             factor(target_author_type)p 
                                  FALSE                                   FALSE 
                  is_author_type_shared                 is_heh_condition_shared 
                                  FALSE                                   FALSE 
             target_is_multisite_author              target_is_mixedsite_author 
                                  FALSE                                   FALSE 
                    target_update_count                 target_update_frequency 
                                  FALSE                                   FALSE 
   target_days_since_most_recent_update          target_days_since_first_update 
                                  FALSE                                   FALSE 
attr(,"sup")
character(0)

## Modeling by feature category

In [18]:
start_time <- proc.time()
fs <- list(
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train),
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         | 0, H_train),
  mlogit(is_target ~ factor(target_author_type) 
         + is_author_type_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         | 0, H_train),
  mlogit(is_target ~ target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train),
  mlogit(is_target ~ is_health_condition_shared | 0, H_train)
)
total_time <- proc.time() - start_time
total_time

   user  system elapsed 
350.130  94.039 444.732 

In [37]:
stargazer::stargazer(
  fs,
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                                   is_target                            
                                            (1)          (2)          (3)          (4)          (5)     
--------------------------------------------------------------------------------------------------------
censored_log(target_outdegree, min = 1)  -0.191***    -0.510***                                         
                                          (0.005)      (0.004)                                          
                                                                                                        
target_has_indegree                       0.756***     0.995***                                         
                                          (0.017)      (0.013)                                          
                                                                                                        
censored_log(target_indegree, min = 1)    0.649***    

In [20]:
print(paste("Train accuracy  &  ",
    lapply(fs, function(x) acc(x, H_train)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))
print(paste("Test accuracy   &  ",
    lapply(fs, function(x) acc(x, H_test)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))

[1] "Train accuracy  &   0.7398 & 0.3322 & 0.1098 & 0.6907 & 0.0504  \\"
[1] "Test accuracy   &   0.7727 & 0.324 & 0.0975 & 0.7331 & 0.0488  \\"


In [36]:
stargazer::stargazer(
  head(fs, -1),
  dep.var.caption = "",
  header=TRUE,
  type='latex')


% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Sat, Jan 11, 2020 - 03:54:04 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
\\[-1.8ex] & \multicolumn{4}{c}{is\_target} \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4)\\ 
\hline \\[-1.8ex] 
 censored\_log(target\_outdegree, min = 1) & $-$0.191$^{***}$ & $-$0.510$^{***}$ &  &  \\ 
  & (0.005) & (0.004) &  &  \\ 
  & & & & \\ 
 target\_has\_indegree & 0.756$^{***}$ & 0.995$^{***}$ &  &  \\ 
  & (0.017) & (0.013) &  &  \\ 
  & & & & \\ 
 censored\_log(target\_indegree, min = 1) & 0.649$^{***}$ & 0.674$^{***}$ &  &  \\ 
  & (0.005) & (0.003) &  &  \\ 
  & & & & \\ 
 is\_reciprocal & 20.016$^{***}$ & 13.068$^{***}$ &  &  \\ 
  & (0.460) & (0.174) &  &  \\ 
  & & & & \\ 
 is\_weakly\_connected & 1.767$^{***}$ & 2.454$^{***}$ &  &  \\ 
  & (0.021) & (0.021) &  &  \\ 
  & & & & \\ 
 i

### Activity features only

Breaking things down by activity features, in order to understand why the test acc was so high. It's because of the "time since most recent update" feature, which is highly predictive of initiation.

In [25]:
start_time <- proc.time()
fst <- list(
  mlogit(is_target ~ target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train),
  mlogit(is_target ~ target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train),
  mlogit(is_target ~ target_update_count
         + target_update_frequency
         | 0, H_train),
  mlogit(is_target ~ target_update_count | 0, H_train),
  mlogit(is_target ~ target_update_frequency | 0, H_train),
  mlogit(is_target ~ target_days_since_most_recent_update | 0, H_train),
  mlogit(is_target ~ target_days_since_first_update | 0, H_train)
)
total_time <- proc.time() - start_time
total_time

   user  system elapsed 
193.116  41.652 235.167 

In [26]:
stargazer::stargazer(
  fst,
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                                             is_target                                         
                                         (1)          (2)          (3)          (4)          (5)          (6)          (7)     
-------------------------------------------------------------------------------------------------------------------------------
target_update_count                    0.001***                  0.002***     0.002***                                         
                                      (0.00002)                 (0.00002)    (0.00002)                                         
                                                                                                                               
target_update_frequency                0.004***                 -0.013***                 -0.013***                            
                                       (0.0002)                  (0.0002)                  (0.0002)    

In [28]:
print(paste("Test accuracy   &  ",
    lapply(fst, function(x) acc(x, H_test)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))

[1] "Test accuracy   &   0.7331 & 0.7579 & 0.0625 & 0.057 & 0.0554 & 0.8105 & 0.3281  \\"


In [29]:
print(paste("Train accuracy  &  ",
    lapply(fst, function(x) acc(x, H_train)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))

[1] "Train accuracy  &   0.6907 & 0.7241 & 0.0812 & 0.0754 & 0.0505 & 0.7901 & 0.3123  \\"


In [30]:
start_time <- proc.time()
fstt <- list(
  mlogit(is_target ~ target_days_since_most_recent_update * target_days_since_first_update
         | 0, H_train)
)
total_time <- proc.time() - start_time
total_time

   user  system elapsed 
 36.109   8.374  44.633 

In [31]:
stargazer::stargazer(
  fstt,
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                                             is_target         
-----------------------------------------------------------------------------------------------
target_days_since_most_recent_update                                         -0.013***         
                                                                             (0.0001)          
                                                                                               
target_days_since_first_update                                               -0.001***         
                                                                             (0.00001)         
                                                                                               
target_days_since_most_recent_update:target_days_since_first_update         0.00000***         
                                                                             (0.00000)         
                                       

In [32]:
print(paste("Train accuracy  &  ",
    lapply(fstt, function(x) acc(x, H_train)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))
print(paste("Test accuracy   &  ",
    lapply(fstt, function(x) acc(x, H_test)) %>% unlist() %>% round(4) %>% paste(collapse=' & '), ' \\'))

[1] "Train accuracy  &   0.7237  \\"
[1] "Test accuracy   &   0.7574  \\"


## First initiation vs All initiations

In [17]:
# load the training data with first initiations only
fn <- '/home/srivbane/shared/caringbridge/data/projects/sna-social-support/author_initiations/author_initiation_choices_train_firstonly.csv'
DM_train_firstonly <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddi')
DM_train_firstonly$is_friend_of_friend = DM_train_firstonly$is_friend_of_friend - DM_train_firstonly$is_reciprocal
DM_train_firstonly$hcf = relevel(factor(DM_train_firstonly$target_health_condition), ref="None")
H_train_firstonly <- DM_train_firstonly %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')

nrow(H_train_firstonly)

Warning message:
“Setting row names on a tibble is deprecated.”


[1] 1665400

In [18]:
# load the test data with first initiations only
fn <- '/home/srivbane/shared/caringbridge/data/projects/sna-social-support/author_initiations/author_initiation_choices_test_firstonly.csv'
DM_test_firstonly <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddi')
DM_test_firstonly$is_friend_of_friend = DM_test_firstonly$is_friend_of_friend - DM_test_firstonly$is_reciprocal
DM_test_firstonly$hcf = relevel(factor(DM_test_firstonly$target_health_condition), ref="None")
H_test_firstonly <- DM_test_firstonly %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')
nrow(H_test_firstonly)

Warning message:
“Setting row names on a tibble is deprecated.”


[1] 203500

In [19]:
start_time <- proc.time()
firstonly_mds <- list(
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train_firstonly),
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train)
)
total_time <- proc.time() - start_time
total_time

   user  system elapsed 
209.982  99.273 139.771 

In [20]:
stargazer::stargazer(
  firstonly_mds,
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                 is_target          
                                             (1)            (2)     
--------------------------------------------------------------------
censored_log(target_outdegree, min = 1)   -0.220***      -0.191***  
                                           (0.009)        (0.005)   
                                                                    
target_has_indegree                        0.775***      0.756***   
                                           (0.025)        (0.017)   
                                                                    
censored_log(target_indegree, min = 1)     0.480***      0.649***   
                                           (0.008)        (0.005)   
                                                                    
is_reciprocal                             30.524***      20.016***  
                                           (1.059)        (0.460)   
                                 

In [21]:
# Train accuracy
acc(firstonly_mds[[1]], H_train_firstonly)
acc(firstonly_mds[[2]], H_train)

[1] 0.7775459

[1] 0.7397915

In [22]:
# Test accuracy
acc(firstonly_mds[[1]], H_test_firstonly)
acc(firstonly_mds[[2]], H_test)

[1] 0.8063882

[1] 0.7727256

## Health condition analysis

In [63]:
# load the training data with non-none initiator health condition only
fn <- '/home/srivbane/shared/caringbridge/data/projects/sna-social-support/author_initiations/author_initiation_choices_train_hcNotNone.csv'
DM_train_hcNotNone <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddic')
DM_train_hcNotNone$is_friend_of_friend = DM_train_hcNotNone$is_friend_of_friend - DM_train_hcNotNone$is_reciprocal
DM_train_hcNotNone$hcf = relevel(factor(DM_train_hcNotNone$target_health_condition), ref="None")
H_train_hcNotNone <- DM_train_hcNotNone %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')
nrow(H_train_hcNotNone)

Warning message:
“Setting row names on a tibble is deprecated.”


[1] 2397775

In [34]:
# health condition model without interactions
# comparing the subset who do and don't have 
md_all <- mlogit(is_target ~ hcf + is_health_condition_shared | 0, H_train)
md_hcNotNone <- mlogit(is_target ~ hcf + is_health_condition_shared | 0, H_train_hcNotNone)
stargazer::stargazer(
  list(md_all, md_hcNotNone),
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                       is_target          
                                   (1)            (2)     
----------------------------------------------------------
hcfCancer                       -0.156***      -0.391***  
                                 (0.006)        (0.011)   
                                                          
hcfCardiovascular/Stroke        -0.117***      -0.222***  
                                 (0.014)        (0.018)   
                                                          
hcfCondition Unknown            -2.739***      -2.785***  
                                 (0.108)        (0.136)   
                                                          
hcfCongenital/Immune Disorder    0.379***      0.399***   
                                 (0.027)        (0.033)   
                                                          
hcfInfant/Childbirth            -0.533***      -0.588***  
                                 (0.021)        (0.026)

In [66]:
md_all <- mlogit(is_target ~ hcf + is_health_condition_shared : hcf | 0, H_train)

In [67]:
stargazer::stargazer(
  md_all,
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                               is_target         
---------------------------------------------------------------------------------
hcfCancer                                                      -0.308***         
                                                                (0.010)          
                                                                                 
hcfCardiovascular/Stroke                                       -0.228***         
                                                                (0.015)          
                                                                                 
hcfCondition Unknown                                           -2.849***         
                                                                (0.109)          
                                                                                 
hcfCongenital/Immune Disorder                                  0.252***          
               

In [68]:
H_train$is_health_condition_provided = ifelse(H_train$hcf == "None", 0, 1)

In [75]:
# Providing a health condition doesn't result in more connections
# But, this model doesn't account for the baseline difference in reporting
# In other words, as < 50% of sites report, of course it doesn't help!  So the true effect, if it IS positive, must be less than the amount necessary to see a positive effect despite the 60/40 recording split
md <- mlogit(is_target ~ is_health_condition_provided | 0, H_train)
stargazer::stargazer(
  md,
  dep.var.caption = "",
  header=TRUE,
  type='text')


                                   is_target         
-----------------------------------------------------
is_heh_condition_provided          -0.423***         
                                    (0.005)          
                                                     
-----------------------------------------------------
Observations                        155,141          
Log Likelihood                   -496,099.900        
Note:                     *p<0.1; **p<0.05; ***p<0.01


In [42]:
nrow(H_train_hcNotNone) / 25
90000 * 25

[1] 95911

[1] 2250000

In [60]:
#levels(H_train_hcNotNone$hcf) <- c("None", "Cancer", "Cardiovascular/Stroke", "Condition Unknown", "Congenital/Immune Disorder", "Infant/Childbirth", 'Injury', 'Neurological Condition', 'Other', 'Surgery/Transplantation')
 levels(H_train_hcNotNone$hcf) <- c("None", "Cancer", "Other", "Other", "Other", "Other", 'Other', 'Other', 'Other', 'Other')
levels(H_train_hcNotNone$hcf)

[1] "None"   "Cancer" "Other"

In [65]:
#H_train_hcNotNone_subset <- H_train_hcNotNone %>% filter(is_health_condition_shared == 1)
#md_hcNotNone <- mlogit(is_target ~ hcf + is_health_condition_shared + is_health_condition_shared : hcf
#                        | 0, H_train_hcNotNone_subset)

In [62]:
stargazer::stargazer(
  md_hcNotNone,
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                 is_target         
---------------------------------------------------
hcfCancer                        -0.345***         
                                  (0.010)          
                                                   
hcfOther                         -0.800***         
                                  (0.009)          
                                                   
is_heh_condition_shared          0.439***          
                                  (0.012)          
                                                   
---------------------------------------------------
Observations                      95,911           
Log Likelihood                 -304,020.000        
Note:                   *p<0.1; **p<0.05; ***p<0.01


In [33]:
stargazer::stargazer(
  list(md_all, md_hcNotNone),
  dep.var.caption = "",
  header=TRUE,
  type='text')    # use type = 'latex' to produce latex table code as the output


                                                               is_target          
                                                           (1)            (2)     
----------------------------------------------------------------------------------
hcfCancer                                               -0.308***      -0.391***  
                                                         (0.010)        (0.011)   
                                                                                  
hcfCardiovascular/Stroke                                -0.228***      -0.222***  
                                                         (0.015)        (0.018)   
                                                                                  
hcfCondition Unknown                                    -2.849***      -2.785***  
                                                         (0.109)        (0.136)   
                                                                                  
hcf

## Geographic analysis

Building models that include geographic features.

In [15]:
# load the training data with geo-identified authors only
fn <- '/home/srivbane/shared/caringbridge/data/projects/sna-social-support/author_initiations/geo_author_initiation_choices_train_all.csv'
DM_train_geo <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddii')
DM_train_geo$is_friend_of_friend = DM_train_geo$is_friend_of_friend - DM_train_geo$is_reciprocal
DM_train_geo$hcf = relevel(factor(DM_train_geo$target_health_condition), ref="None")
H_train_geo <- DM_train_geo %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')
nrow(H_train_geo)

Warning message:
“Setting row names on a tibble is deprecated.”


[1] 175175

In [16]:
# load the test data with geo-identified authors only
fn <- '/home/srivbane/shared/caringbridge/data/projects/sna-social-support/author_initiations/geo_author_initiation_choices_test_all.csv'
DM_test_geo <- read_csv(fn, col_types='iiiiiiiiiiiccciiiidddii')
DM_test_geo$is_friend_of_friend = DM_test_geo$is_friend_of_friend - DM_test_geo$is_reciprocal
DM_test_geo$hcf = relevel(factor(DM_test_geo$target_health_condition), ref="None")
H_test_geo <- DM_test_geo %>%
     group_by(choice_id) %>% mutate(alt_id=row_number()) %>% ungroup() %>%
     mlogit.data(shape="long", chid.var='choice_id', choice="is_target", alt.var='alt_id')
nrow(H_test_geo)

Warning message:
“Setting row names on a tibble is deprecated.”


[1] 16700

In [17]:
start_time <- proc.time()
geo_mds <- list(
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train),
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         | 0, H_train_geo),
  mlogit(is_target ~ censored_log(target_outdegree, min=1) 
         + target_has_indegree 
         + censored_log(target_indegree, min=1) 
         + is_reciprocal
         + is_weakly_connected
         + is_friend_of_friend
         + factor(target_author_type) 
         + is_author_type_shared
         + is_health_condition_shared
         + target_is_multisite_author
         + target_is_mixedsite_author
         + target_update_count
         + target_update_frequency
         + target_days_since_most_recent_update
         + target_days_since_first_update
         + is_state_assignment_shared
         | 0, H_train_geo)
)
total_time <- proc.time() - start_time
total_time

   user  system elapsed 
165.795  20.340 165.593 

In [20]:
stargazer::stargazer(
  geo_mds,
  dep.var.caption = "",
  header=TRUE,
  type='text')


                                                    is_target             
                                            (1)         (2)        (3)    
--------------------------------------------------------------------------
censored_log(target_outdegree, min = 1)  -0.191***   -0.225***  -0.231*** 
                                          (0.005)     (0.025)    (0.029)  
                                                                          
target_has_indegree                       0.756***    1.907***   1.795*** 
                                          (0.017)     (0.170)    (0.182)  
                                                                          
censored_log(target_indegree, min = 1)    0.649***    0.960***   0.974*** 
                                          (0.005)     (0.027)    (0.030)  
                                                                          
is_reciprocal                            20.016***    8.458***   7.999*** 
                        

In [19]:
# The exponentiated coefficient represents the change in the odds
# of being the target vis-a-vis the non-target category,
# associated with having a shared state assignment."

(exp(geo_mds[[3]]$coefficients) - 1) * 100

censored_log(target_outdegree, min = 1)                     target_has_indegree 
                          -2.061099e+01                            5.017288e+02 
 censored_log(target_indegree, min = 1)                           is_reciprocal 
                           1.648973e+02                            2.976714e+05 
                    is_weakly_connected                     is_friend_of_friend 
                           7.131468e+03                            1.545822e+03 
        factor(target_author_type)mixed             factor(target_author_type)p 
                           1.178365e+01                           -2.187840e+01 
                  is_author_type_shared                 is_heh_condition_shared 
                           3.426486e+01                            4.914082e+01 
             target_is_multisite_author              target_is_mixedsite_author 
                           1.164367e+02                            4.216178e+01 
                    target_update_count                 target_update_frequency 
                          -4.497662e-02                            1.267679e+00 
   target_days_since_most_recent_update          target_days_since_first_update 
                          -5.389847e-01                           -4.974286e-02 
             is_state_assignment_shared 
                           1.422443e+03 
attr(,"names.sup.coef")
character(0)
attr(,"fixed")
censored_log(target_outdegree, min = 1)                     target_has_indegree 
                                  FALSE                                   FALSE 
 censored_log(target_indegree, min = 1)                           is_reciprocal 
                                  FALSE                                   FALSE 
                    is_weakly_connected                     is_friend_of_friend 
                                  FALSE                                   FALSE 
        factor(target_author_type)mixed             factor(target_author_type)p 
                                  FALSE                                   FALSE 
                  is_author_type_shared                 is_heh_condition_shared 
                                  FALSE                                   FALSE 
             target_is_multisite_author              target_is_mixedsite_author 
                                  FALSE                                   FALSE 
                    target_update_count                 target_update_frequency 
                                  FALSE                                   FALSE 
   target_days_since_most_recent_update          target_days_since_first_update 
                                  FALSE                                   FALSE 
             is_state_assignment_shared 
                                  FALSE 
attr(,"sup")
character(0)

In [21]:
# Test accuracy
acc(geo_mds[[1]], H_test)
acc(geo_mds[[2]], H_test_geo)
acc(geo_mds[[3]], H_test_geo)

[1] 0.7727256

[1] 0.8428144

[1] 0.8712575

In [22]:
md <- mlogit(is_target ~ is_state_assignment_shared | 0, H_train_geo)
stargazer::stargazer(
  md,
  dep.var.caption = "",
  header=TRUE,
  type='text')
acc(md, H_test_geo)


                                    is_target         
------------------------------------------------------
is_state_assignment_shared          3.139***          
                                     (0.031)          
                                                      
------------------------------------------------------
Observations                          7,007           
Log Likelihood                     -17,409.340        
Note:                      *p<0.1; **p<0.05; ***p<0.01


[1] 0.261976